In [ ]:
import discord
from discord.ext import commands

# Erstelle ein Intents-Objekt und aktiviere die benötigten Intents
intents = discord.Intents.default()
intents.message_content = True  # Ermöglicht den Zugriff auf Nachrichteninhalte

# Erstelle ein Bot-Objekt mit den spezifizierten Intents
bot = commands.Bot(command_prefix='!', intents=intents)

# Event, das ausgeführt wird, wenn der Bot sich mit Discord verbindet
@bot.event
async def on_ready():
    print(f'Bot ist eingeloggt als {bot.user}')

# Definiere den Befehl !hallo
@bot.command()
async def hallo(ctx):
    await ctx.send('Hallo!')

# Starte den Bot mit deinem Token
bot.run('')

In [ ]:
import discord
from discord.ext import commands
from dotenv import load_dotenv
import os
import logging

# Lade Umgebungsvariablen aus der .env Datei
load_dotenv()

# Konfiguriere das Logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('discord_bot.log'),  # Logs werden in dieser Datei gespeichert
        logging.StreamHandler()  # Logs werden weiterhin auch auf der Konsole ausgegeben
    ]
)
logger = logging.getLogger('discord_bot')

# Bot-Token und Guild-ID aus Umgebungsvariablen laden
bot_token = os.getenv('DISCORD_BOT_TOKEN')
allowed_guild_id = int(os.getenv('ALLOWED_GUILD_ID'))

# Erstelle ein Intents-Objekt und aktiviere die benötigten Intents
intents = discord.Intents.default()
intents.message_content = True  # Ermöglicht den Zugriff auf Nachrichteninhalte

# Erstelle ein Bot-Objekt mit den spezifizierten Intents
bot = commands.Bot(command_prefix='!', intents=intents)

# Event, das ausgeführt wird, wenn der Bot sich mit Discord verbindet
@bot.event
async def on_ready():
    logger.info(f'Bot ist eingeloggt als {bot.user}')
    logger.info('Bot ist aktiv auf folgenden Servern:')
    for guild in bot.guilds:
        logger.info(f'- {guild.name} (ID: {guild.id})')

# Überprüfe, ob der Bot in einer erlaubten Gilde (Server) ist
@bot.event
async def on_guild_join(guild):
    if guild.id != allowed_guild_id:
        await guild.leave()
        logger.warning(f'Bot hat den unerlaubten Server {guild.name} (ID: {guild.id}) verlassen.')

# Definiere den Befehl !hallo
@bot.command()
async def hallo(ctx):
    if ctx.guild.id == allowed_guild_id:
        await ctx.send('Hallo!')
        logger.info(f'Befehl !hallo wurde von {ctx.author} im Channel {ctx.channel} auf dem Server {ctx.guild.name} ausgeführt.')
    else:
        await ctx.send('Dieser Bot ist nicht für diesen Server zugelassen.')
        logger.warning(f'Unberechtigter Befehl !hallo von {ctx.author} im Channel {ctx.channel} auf dem Server {ctx.guild.name}.')

# Definiere den Befehl !frage
@bot.command()
async def frage(ctx, *, text: str):
    if ctx.guild.id == allowed_guild_id:
        print(f'Frage von {ctx.author}: {text}')
        await ctx.send(f'Deine Frage wurde im Terminal ausgegeben: {text}')
        logger.info(f'Befehl !frage wurde von {ctx.author} im Channel {ctx.channel} auf dem Server {ctx.guild.name} ausgeführt.')
    else:
        await ctx.send('Dieser Bot ist nicht für diesen Server zugelassen.')
        logger.warning(f'Unberechtigter Befehl !frage von {ctx.author} im Channel {ctx.channel} auf dem Server {ctx.guild.name}.')

# Starte den Bot mit deinem Token
bot.run(bot_token)

In [ ]:
import discord
from discord.ext import commands
from dotenv import load_dotenv
import os
import logging
import subprocess


# Lade Umgebungsvariablen aus der .env Datei
load_dotenv()

# Konfiguriere das Logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('discord_bot.log'),  # Logs werden in dieser Datei gespeichert
        logging.StreamHandler()  # Logs werden weiterhin auch auf der Konsole ausgegeben
    ]
)
logger = logging.getLogger('discord_bot')

# Bot-Token und Guild-ID aus Umgebungsvariablen laden
bot_token = os.getenv('DISCORD_BOT_TOKEN')
allowed_guild_id = int(os.getenv('ALLOWED_GUILD_ID'))

# Erstelle ein Intents-Objekt und aktiviere die benötigten Intents
intents = discord.Intents.default()
intents.message_content = True  # Ermöglicht den Zugriff auf Nachrichteninhalte

# Erstelle ein Bot-Objekt mit den spezifizierten Intents
bot = commands.Bot(command_prefix='!', intents=intents)

# Event, das ausgeführt wird, wenn der Bot sich mit Discord verbindet
@bot.event
async def on_ready():
    logger.info(f'Bot ist eingeloggt als {bot.user}')
    logger.info('Bot ist aktiv auf folgenden Servern:')
    for guild in bot.guilds:
        logger.info(f'- {guild.name} (ID: {guild.id})')

# Überprüfe, ob der Bot in einer erlaubten Gilde (Server) ist
@bot.event
async def on_guild_join(guild):
    if guild.id != allowed_guild_id:
        await guild.leave()
        logger.warning(f'Bot hat den unerlaubten Server {guild.name} (ID: {guild.id}) verlassen.')

# Definiere den Befehl !hallo
@bot.command()
async def hallo(ctx):
    if ctx.guild.id == allowed_guild_id:
        await ctx.send('Hallo!')
        logger.info(f'Befehl !hallo wurde von {ctx.author} im Channel {ctx.channel} auf dem Server {ctx.guild.name} ausgeführt.')
    else:
        await ctx.send('Dieser Bot ist nicht für diesen Server zugelassen.')
        logger.warning(f'Unberechtigter Befehl !hallo von {ctx.author} im Channel {ctx.channel} auf dem Server {ctx.guild.name}.')

@bot.command()
async def frage(ctx, *, text: str):
    if ctx.guild.id == allowed_guild_id:
        # Zuerst eine Bestätigungsnachricht senden
        await ctx.send('Frage wird beantwortet...')
        
        # Das andere Skript "test.py" ausführen und die Antwort zurückgeben
        result = subprocess.run(['python3', 'test.py', text], capture_output=True, text=True)
        
        antwort = result.stdout.strip()  # Die Ausgabe des Skripts erfassen

        # Nachricht im gewünschten Format senden
        formatted_response = f"Frage: {text}\nAntwort: {antwort}"
        await ctx.send(formatted_response)
        
        # Protokollierung der Frage und der Antwort
        logger.info(f'Frage von {ctx.author}: {text}')
        logger.info(f'Antwort: {antwort}')
        
        logger.info(f'Befehl !frage wurde von {ctx.author} im Channel {ctx.channel} auf dem Server {ctx.guild.name} ausgeführt.')
    else:
        await ctx.send('Dieser Bot ist nicht für diesen Server zugelassen.')
        logger.warning(f'Unberechtigter Befehl !frage von {ctx.author} im Channel {ctx.channel} auf dem Server {ctx.guild.name}.')

# Starte den Bot mit deinem Token
bot.run(bot_token)